##### Copyright 2020 Google

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Routing with t|ket>

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/experiments/qaoa/routing_with_tket"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/ReCirq/blob/master/docs/qaoa/routing_with_tket.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/ReCirq/blob/master/docs/qaoa/routing_with_tket.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/ReCirq/docs/qaoa/routing_with_tket.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

Wrap tket's compilation unit framework to keep track of qubit mappings and work with generic devices.

## Setup

Install the ReCirq package:

In [ ]:
try:
    import recirq
except ImportError:
    !pip install git+https://github.com/quantumlib/ReCirq

Now import Cirq, ReCirq and the module dependencies:

In [ ]:
import cirq
import recirq
import networkx as nx
from cirq.contrib.svg import SVGCircuit
import numpy as np

In [ ]:
from pytket.predicates import CompilationUnit, ConnectivityPredicate
from pytket.passes import SequencePass, RoutingPass, DecomposeSwapsToCXs
from pytket.routing import GraphPlacement

### Example circuit
We'll route a 3-regular circuit to Sycamore23. To try to clear up some of the confusion about which indices are which, we'll construct the initial circuit with `LineQubits` 10 through 19 which should be thought of as "logical indices".

In [ ]:
from recirq.qaoa.problem_circuits import get_generic_qaoa_circuit
from recirq.qaoa.gates_and_compilation import compile_problem_unitary_to_arbitrary_zz, \
    compile_driver_unitary_to_rx

problem_graph = nx.random_regular_graph(d=3, n=10)
nx.set_edge_attributes(problem_graph, values=1, name='weight')
circuit_qubits = cirq.LineQubit.range(10, 20)
gammas = np.random.randn(2)
betas = np.random.randn(2)
circuit = get_generic_qaoa_circuit(
    problem_graph=problem_graph,
    qubits=circuit_qubits,
    gammas=gammas,
    betas=betas)
circuit = compile_problem_unitary_to_arbitrary_zz(circuit)
circuit = compile_driver_unitary_to_rx(circuit)
SVGCircuit(circuit)

### "Route" this circuit

Let's look at the "connectivity graph" of the circuit vs. that of the device

In [ ]:
import cirq.contrib.routing as ccr

uncompiled_c_graph = ccr.get_circuit_connectivity(circuit)
nx.draw_networkx(uncompiled_c_graph)

In [ ]:
import cirq.google as cg

dev_graph = ccr.xmon_device_to_graph(cg.Sycamore23)
nx.draw_networkx(dev_graph)

In [ ]:
# alias for the device. If this notebook were wrapped
# in a function, `circuit` and `device` would be the arguments
device = cg.Sycamore23

### Convert to pytket `Device`
The provided function doesn't work with `SerializableDevice`. We use existing functionality to turn Devices into graphs to provide a more robust solution.

In [ ]:
import pytket
from pytket.circuit import Node

def _qubit_index_edges():
    dev_graph = ccr.xmon_device_to_graph(device)
    for n1, n2 in dev_graph.edges:
        yield Node('grid', n1.row, n1.col), Node('grid', n2.row, n2.col)

def _device_to_tket_device():
    arc = pytket.routing.Architecture(
        list(_qubit_index_edges())
    )
    return pytket.device.Device({}, {}, arc)

tk_circuit = pytket.cirq.cirq_to_tk(circuit)
tk_device = _device_to_tket_device()

In [ ]:
tk_circuit.qubits

In [ ]:
tk_device.coupling

### Placement and routing pass

In [ ]:
from pytket.predicates import CompilationUnit, ConnectivityPredicate
from pytket.passes import SequencePass, RoutingPass, DecomposeSwapsToCXs, PlacementPass
from pytket.routing import GraphPlacement

In [ ]:
unit = CompilationUnit(tk_circuit, [ConnectivityPredicate(tk_device)])
passes = SequencePass([
    PlacementPass(GraphPlacement(tk_device)),
    RoutingPass(tk_device)])
passes.apply(unit)
valid = unit.check_all_predicates()
assert valid

### The initial mapping
This maps from logical LineQubits to "physical" GridQubits

In [ ]:
unit.initial_map

### Bookkept initial mapping
We "decode" our tket conventions back into Cirq idioms.

In [ ]:
def tk_to_cirq_qubit(tk):
    ind = tk.index
    return cirq.LineQubit(ind[0]) if len(ind) == 1 else cirq.GridQubit(*ind)

initial_map = {tk_to_cirq_qubit(n1): tk_to_cirq_qubit(n2) for n1, n2 in unit.initial_map.items()}
initial_map

### The final mapping
This maps from logical LineQubits to final GridQubits

In [ ]:
unit.final_map

In [ ]:
final_map = {tk_to_cirq_qubit(n1): tk_to_cirq_qubit(n2)
             for n1, n2 in unit.final_map.items()}
final_map

### The compilation unit applies the mapping
So our circuit qubits are now GridQubits

In [ ]:
unit.circuit.qubits

### Convert the circuit back to Cirq

In [ ]:
routed_circuit = pytket.cirq.tk_to_cirq(unit.circuit)
SVGCircuit(routed_circuit)

### Now it's nice and compiled

In [ ]:
routed_c_graph = ccr.get_circuit_connectivity(routed_circuit)
nx.draw_networkx(routed_c_graph)

### Check that circuits are equivalent

In [ ]:
for _, op, _ in routed_circuit.findall_operations_with_gate_type(cirq.TwoQubitGate):
    a, b = op.qubits
    assert a.is_adjacent(b)

In [ ]:
import cirq.contrib.acquaintance as cca
def permute_gate(qubits, permutation):
    return cca.LinearPermutationGate(
        num_qubits=len(qubits),
        permutation={i: permutation[i] for i in range(len(permutation))}
    ).on(*qubits)

final_to_initial_map = {final_map[cq]: initial_map[cq]
                              for cq in circuit_qubits}
initial_qubits = [initial_map[cq] for cq in circuit_qubits]
final_permutation = [initial_qubits.index(final_to_initial_map[q])
                     for q in initial_qubits]
rcircuit_with_perm = routed_circuit.copy()
rcircuit_with_perm.append(permute_gate(initial_qubits, final_permutation))
expected = circuit.unitary(qubit_order=cirq.QubitOrder.explicit(circuit_qubits))
actual = rcircuit_with_perm.unitary(qubit_order=cirq.QubitOrder.explicit(initial_qubits))
cirq.testing.assert_allclose_up_to_global_phase(expected, actual, atol=1e-8)